Ajude **Santander** a identificar o valor das transações para cada cliente potencial. Esse é um primeiro passo que o **Santander** precisa acertar para personalizar seus serviços em grande escala.

Baixa o dataset *train* no url seguinte:  
https://www.kaggle.com/c/santander-value-prediction-challenge/data

Depois, pode deixar ele na pasta `colab_data` do `Drive` (com um *Drag and Drop*)

O case podera ser quebrado nas **6** partes seguintes:  
- **Identificar o problema**
  - Qual o tipo (classificaçao, regressao, clustering) de problema e porque? 
- **Necessidades de aplicar transformaçoes?**
  - Ex: *imputing* de valores null, *encoding* de colunas *string*, etc 
- **Separar os sets de treinamento e teste**
- **Baseline**
  - Achar uma baseline, um primeiro modelo para ter uma referencia
- **Escolher a metrica**
- **Melhorar o resultado**
  - Feature engineering, otimizaçao do modelo, hiperparametros, etc

In [2]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Rafael Pietro\OneDrive\Cursos\DNC\Datasets\Modelos Supervisionados\Case\train.csv")
df.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: float64(1845), int64(3147), object(1)
memory usage: 169.9+ MB


In [4]:
df["target"].describe()

count    4.459000e+03
mean     5.944923e+06
std      8.234312e+06
min      3.000000e+04
25%      6.000000e+05
50%      2.260000e+06
75%      8.000000e+06
max      4.000000e+07
Name: target, dtype: float64

In [5]:
id = df["ID"]
y = df["target"]
X = df.drop(["ID", "target"], axis=1)

In [6]:
print(id.shape,'\n',y.shape,'\n',X.shape)

(4459,) 
 (4459,) 
 (4459, 4991)


In [7]:
df.corr()['target'].abs().sort_values(ascending=False)

## Split train test

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [32]:
print(X_train.shape,'\n',X_test.shape,'\n',y_train.shape,'\n',y_test.shape)

(3567, 4991) 
 (892, 4991) 
 (3567,) 
 (892,)


## Baseline

In [33]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

In [25]:
from sklearn.metrics import mean_absolute_error

y_train_pred = reg.predict(X_train)
y_pred = reg.predict(X_test)
print(mean_absolute_error(y_train, y_train_pred))
print(mean_absolute_error(y_test, y_pred))

13930.62449721572
3176573602530918.5


In [34]:
from sklearn.feature_selection import SelectKBest, f_regression
sel_kbest = SelectKBest(f_regression, k=45).fit(X_train, y_train)

c:\Anaconda\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [36]:
sel_kbest

SelectKBest(k=45, score_func=<function f_regression at 0x0000024C88177160>)

In [35]:
X_train_sel = sel_kbest.transform(X_train)
X_test_sel = sel_kbest.transform(X_test)

In [37]:
X_train_sel.shape

(3567, 45)

In [42]:
X_train_sel

array([[       0.  ,        0.  ,        0.  , ...,        0.  ,
               0.  ,        0.  ],
       [47500000.  , 20000000.  , 36000000.  , ..., 30000000.  ,
        48571428.58, 20000000.  ],
       [       0.  ,        0.  ,        0.  , ...,        0.  ,
               0.  ,        0.  ],
       ...,
       [       0.  ,        0.  ,        0.  , ...,        0.  ,
               0.  ,        0.  ],
       [  400000.  ,   350000.  ,   400000.  , ...,        0.  ,
          400000.  ,        0.  ],
       [  200000.  ,        0.  ,        0.  , ...,        0.  ,
         2000000.  ,        0.  ]])

In [43]:
reg = LinearRegression().fit(X_train_sel, y_train)

y_train_pred = reg.predict(X_train_sel)
y_pred = reg.predict(X_test_sel)

print(mean_absolute_error(y_train, y_train_pred))
print(mean_absolute_error(y_test, y_pred))

5322548.588998504
5230263.099091365


## Escolher a metrica

In [44]:
from sklearn.metrics import mean_squared_log_error

In [45]:
y_train_pred = reg.predict(X_train_sel)
y_pred = reg.predict(X_test_sel)

In [46]:
y_train = y_train[y_train_pred > 0]
y_train_pred = y_train_pred[y_train_pred > 0]
y_test = y_test[y_pred > 0]
y_pred = y_pred[y_pred > 0]

In [47]:
print(mean_squared_log_error(y_train, y_train_pred))
print(mean_squared_log_error(y_test, y_pred))

3.911277869992664
3.7452270716462412


## Melhorar o resultado

In [48]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [49]:
sel_kbest = SelectKBest(f_regression, k=45).fit(X_train, y_train)
X_train_sel = sel_kbest.transform(X_train)
X_test_sel = sel_kbest.transform(X_test)

c:\Anaconda\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [50]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=6, random_state=0)
regr.fit(X_train_sel, y_train)

RandomForestRegressor(max_depth=6, random_state=0)

In [51]:
y_train_pred = regr.predict(X_train_sel)
y_pred = regr.predict(X_test_sel)


print(mean_squared_log_error(y_train, y_train_pred))
print(mean_squared_log_error(y_test, y_pred))

3.832475327822927
3.719739927585444


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'max_depth':[2, 6, 10, 20], 'n_estimators':[20, 50, 100, 200]}
regr = RandomForestRegressor(random_state=0, n_jobs=-1)

clf = GridSearchCV(regr, parameters)
clf.fit(X_train_sel, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             oob_score=False, random_state=0,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs

In [ ]:
y_train_pred = clf.predict(X_train_sel)
y_pred = clf.predict(X_test_sel)


print(mean_squared_log_error(y_train, y_train_pred))
print(mean_squared_log_error(y_test, y_pred))

2.6419218238697284
3.2293333350212725


In [ ]:
clf.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)